In [39]:
from sklearn.ensemble import IsolationForest

from ensemble_model import AnomalyEnsembleMonad
import pandas as pd

In [40]:
data = pd.read_csv("src/test_data.csv")

In [41]:
data

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,2024-11-04 08:08:08
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,2024-11-04 08:09:35
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,2024-11-04 08:07:04
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,2024-11-04 08:09:06
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,2024-11-04 08:06:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2507,TX002508,AC00297,856.21,2023-04-26 17:09:36,Credit,Colorado Springs,D000625,21.157.41.17,M072,Branch,33,Doctor,109,1,12690.79,2024-11-04 08:11:29
2508,TX002509,AC00322,251.54,2023-03-22 17:36:48,Debit,Tucson,D000410,49.174.157.140,M029,Branch,48,Doctor,177,1,254.75,2024-11-04 08:11:42
2509,TX002510,AC00095,28.63,2023-08-21 17:08:50,Debit,San Diego,D000095,58.1.27.124,M087,Branch,56,Retired,146,1,3382.91,2024-11-04 08:08:39
2510,TX002511,AC00118,185.97,2023-02-24 16:24:46,Debit,Denver,D000634,21.190.11.223,M041,Online,23,Student,19,1,1776.91,2024-11-04 08:12:22


In [42]:
# df_encoded = pd.get_dummies(data, columns= ["TransactionType","Location"], dtype="float64")
df_encoded = data
df_encoded["TransactionType"] = data["TransactionType"].map(data["TransactionType"].value_counts())
df_encoded["Location"] = data["Location"].map(data["Location"].value_counts())


In [43]:
features = df_encoded.columns


In [44]:
features

Index(['TransactionID', 'AccountID', 'TransactionAmount', 'TransactionDate',
       'TransactionType', 'Location', 'DeviceID', 'IP Address', 'MerchantID',
       'Channel', 'CustomerAge', 'CustomerOccupation', 'TransactionDuration',
       'LoginAttempts', 'AccountBalance', 'PreviousTransactionDate'],
      dtype='object')

In [45]:
def get_encoded_features(columns, key):
    required_features = []
    for f in columns:
        if f.startswith(key):
            required_features.append(f)
    return required_features


In [46]:
required_features = []
# required_features += get_encoded_features(features, "Location_")
# required_features += get_encoded_features(features, "TransactionType_")


In [47]:
required_features += ["TransactionAmount","TransactionID","TransactionType","Location"]

In [48]:
input_features = df_encoded[required_features]

In [49]:
monad = AnomalyEnsembleMonad(input_features, ["TransactionAmount","TransactionType","Location"])

In [54]:
from ensemble_model import IsolationForestModel

model = IsolationForest(
    n_estimators=100,
    max_samples=2000,
)
iso = IsolationForestModel(model)

(monad.bind(iso.process))

/home/prayash/dev/Ensemble-Anomaly-Detection/src/ensemble_model.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomaly_monad.data_frame[self.column_name] = self.model.predict(features_matrix)


In [55]:
monad.data_frame

,TransactionAmount,TransactionID,TransactionType,Location,is_anomaly
0,14.09,TX000001,1944,59,1
1,376.24,TX000002,1944,63,1
2,126.29,TX000003,1944,61,1
3,184.50,TX000004,1944,59,1
4,13.45,TX000005,568,61,-1
...,...,...,...,...,...
2507,856.21,TX002508,568,60,-1
2508,251.54,TX002509,1944,67,1
2509,28.63,TX002510,1944,59,1
2510,185.97,TX002511,1944,62,1
